In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn as sk

# Carregando a Base de Dados

https://www.kaggle.com/zalando-research/fashionmnist

In [2]:
dataset_train = pd.read_csv("fashion_mnist/train.csv")
dataset_test = pd.read_csv("fashion_mnist/test.csv")

dataset_train

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Pré-processamento da base de dados

1. Separação dos dados em X (variáveis independentes) e y (variável dependente)
2. Normalização

In [3]:
X_train = dataset_train.iloc[:, 1:]
y_train = dataset_train.iloc[:, 0]
X_test = dataset_test.iloc[:, 1:]
y_test = dataset_test.iloc[:, 0]

X_train

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,5,0,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,1,2,0,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,0,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,0,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train

array([[-0.00950488, -0.02269303, -0.02890692, ..., -0.15825831,
        -0.09143181, -0.03304557],
       [-0.00950488, -0.02269303, -0.02890692, ..., -0.15825831,
        -0.09143181, -0.03304557],
       [-0.00950488, -0.02269303, -0.02890692, ..., -0.15825831,
        -0.09143181, -0.03304557],
       ...,
       [-0.00950488, -0.02269303, -0.02890692, ..., -0.15825831,
        -0.09143181, -0.03304557],
       [-0.00950488, -0.02269303, -0.02890692, ..., -0.15825831,
        -0.09143181, -0.03304557],
       [-0.00950488, -0.02269303, -0.02890692, ..., -0.15825831,
        -0.09143181, -0.03304557]])

# Arquitetura de Rede Neural (MLP)

In [5]:
tf.__version__

'2.6.0'

In [6]:
gpus = tf.config.list_physical_devices('GPU')

gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

# GridSearch na MLP

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [8]:
def create_model(optimizer='adam'):
    ann = Sequential()
    ann.add(Dense(units=128, activation='relu', kernel_initializer='he_normal'))
    ann.add(Dense(units=10, activation='softmax'))
    ann.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', 
                metrics=['accuracy'])
    
    return ann

In [9]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

In [10]:
# define the grid search parameters
optimizer = ['SGD', 'Adam']
batch_size = [16, 32]
epochs = [20, 30]
param_grid = dict(optimizer=optimizer, batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=None, cv=5)
grid_result = grid.fit(X_train, y_train)

Epoch 1/20
3000/3000 [==============================] - 8s 2ms/step - loss: 0.5058 - accuracy: 0.8219
Epoch 2/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3734 - accuracy: 0.8665
Epoch 3/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3337 - accuracy: 0.8798
Epoch 4/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3083 - accuracy: 0.8887
Epoch 5/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.2898 - accuracy: 0.8957
Epoch 6/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.2741 - accuracy: 0.9011
Epoch 7/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.2592 - accuracy: 0.9072
Epoch 8/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.2465 - accuracy: 0.9115
Epoch 9/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.2347 - accuracy: 0.9155
Epoch 10/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.224

3000/3000 [==============================] - 5s 2ms/step - loss: 0.1624 - accuracy: 0.9436
Epoch 18/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.1547 - accuracy: 0.9471
Epoch 19/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.1500 - accuracy: 0.9473
Epoch 20/20
750/750 [==============================] - 1s 1ms/step - loss: 0.3538 - accuracy: 0.8896
Epoch 1/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.5041 - accuracy: 0.8230
Epoch 2/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3719 - accuracy: 0.8670
Epoch 3/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3345 - accuracy: 0.8808
Epoch 4/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3083 - accuracy: 0.8892
Epoch 5/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.2874 - accuracy: 0.8972
Epoch 6/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.2717 - accurac

3000/3000 [==============================] - 6s 2ms/step - loss: 0.1681 - accuracy: 0.9396
Epoch 14/20
3000/3000 [==============================] - 6s 2ms/step - loss: 0.1582 - accuracy: 0.9433
Epoch 15/20
3000/3000 [==============================] - 6s 2ms/step - loss: 0.1600 - accuracy: 0.9429
Epoch 16/20
3000/3000 [==============================] - 6s 2ms/step - loss: 0.1485 - accuracy: 0.9466
Epoch 17/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.1367 - accuracy: 0.9505
Epoch 18/20
3000/3000 [==============================] - 6s 2ms/step - loss: 0.1436 - accuracy: 0.9505
Epoch 19/20
3000/3000 [==============================] - 5s 2ms/step - loss: 0.1288 - accuracy: 0.9538
Epoch 20/20
750/750 [==============================] - 1s 1ms/step - loss: 0.5776 - accuracy: 0.8852
Epoch 1/20
3000/3000 [==============================] - 6s 2ms/step - loss: 0.4814 - accuracy: 0.8334
Epoch 2/20
3000/3000 [==============================] - 6s 2ms/step - loss: 0.3558 - acc

3000/3000 [==============================] - 5s 2ms/step - loss: 0.1022 - accuracy: 0.9680
Epoch 30/30
750/750 [==============================] - 1s 1ms/step - loss: 0.3920 - accuracy: 0.8809
Epoch 1/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.5019 - accuracy: 0.8237
Epoch 2/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3710 - accuracy: 0.8689
Epoch 3/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3335 - accuracy: 0.8811
Epoch 4/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3085 - accuracy: 0.8893
Epoch 5/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.2889 - accuracy: 0.8957
Epoch 6/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.2727 - accuracy: 0.9015
Epoch 7/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.2583 - accuracy: 0.9086
Epoch 8/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.2458 - accuracy:

Epoch 16/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.1723 - accuracy: 0.9399
Epoch 17/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.1650 - accuracy: 0.9425
Epoch 18/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.1580 - accuracy: 0.9446
Epoch 19/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.1518 - accuracy: 0.9475
Epoch 20/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.1447 - accuracy: 0.9498
Epoch 21/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.1403 - accuracy: 0.9515
Epoch 22/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.1342 - accuracy: 0.9535
Epoch 23/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.1289 - accuracy: 0.9567
Epoch 24/30
3000/3000 [==============================] - 5s 2ms/step - loss: 0.1241 - accuracy: 0.9582
Epoch 25/30
3000/3000 [==============================] - 5s 2ms/step - lo

...

1500/1500 [==============================] - 3s 2ms/step - loss: 0.2059 - accuracy: 0.9237
Epoch 9/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1955 - accuracy: 0.9269
Epoch 10/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1824 - accuracy: 0.9334
Epoch 11/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1716 - accuracy: 0.9367
Epoch 12/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1618 - accuracy: 0.9407
Epoch 13/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1501 - accuracy: 0.9446
Epoch 14/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1493 - accuracy: 0.9460
Epoch 15/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1347 - accuracy: 0.9501: 0s - loss: 0.1311 
Epoch 16/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1295 - accuracy: 0.9520
Epoch 17/20
1500/1500 [==============================] - 3s 2ms/st

1500/1500 [==============================] - 3s 2ms/step - loss: 0.3207 - accuracy: 0.8850
Epoch 6/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3064 - accuracy: 0.8887
Epoch 7/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2942 - accuracy: 0.8951
Epoch 8/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2831 - accuracy: 0.8992
Epoch 9/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2733 - accuracy: 0.9010
Epoch 10/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2643 - accuracy: 0.9050
Epoch 11/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2559 - accuracy: 0.9078
Epoch 12/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2478 - accuracy: 0.9103
Epoch 13/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2403 - accuracy: 0.9133
Epoch 14/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2327 - acc

1500/1500 [==============================] - 3s 2ms/step - loss: 0.1932 - accuracy: 0.9328
Epoch 22/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1883 - accuracy: 0.9350
Epoch 23/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1833 - accuracy: 0.9376
Epoch 24/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1784 - accuracy: 0.9384
Epoch 25/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1735 - accuracy: 0.9405
Epoch 26/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1699 - accuracy: 0.9411
Epoch 27/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1659 - accuracy: 0.9437
Epoch 28/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1614 - accuracy: 0.9454
Epoch 29/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1569 - accuracy: 0.9475
Epoch 30/30
375/375 [==============================] - 1s 1ms/step - loss: 0.3573 - a

1500/1500 [==============================] - 3s 2ms/step - loss: 0.2205 - accuracy: 0.9190
Epoch 8/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2057 - accuracy: 0.9246
Epoch 9/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1936 - accuracy: 0.9286
Epoch 10/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1868 - accuracy: 0.9310
Epoch 11/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1735 - accuracy: 0.9359
Epoch 12/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1628 - accuracy: 0.9410
Epoch 13/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1524 - accuracy: 0.9435
Epoch 14/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1455 - accuracy: 0.9466
Epoch 15/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1407 - accuracy: 0.9493
Epoch 16/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1344 - a

Epoch 25/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0906 - accuracy: 0.9677
Epoch 26/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0844 - accuracy: 0.9694
Epoch 27/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0866 - accuracy: 0.9691
Epoch 28/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0796 - accuracy: 0.9728
Epoch 29/30
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0824 - accuracy: 0.9712
Epoch 30/30
375/375 [==============================] - 1s 1ms/step - loss: 0.7110 - accuracy: 0.8823
Epoch 1/30
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5380 - accuracy: 0.8112
Epoch 2/30
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3929 - accuracy: 0.8609
Epoch 3/30
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3566 - accuracy: 0.8734
Epoch 4/30
1875/1875 [==============================] - 3s 2ms/step - loss: 0.

# Resultados

In [11]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.887717 using {'batch_size': 32, 'epochs': 30, 'optimizer': 'SGD'}
0.887167 (0.003656) with: {'batch_size': 16, 'epochs': 20, 'optimizer': 'SGD'}
0.880950 (0.002661) with: {'batch_size': 16, 'epochs': 20, 'optimizer': 'Adam'}
0.883267 (0.001825) with: {'batch_size': 16, 'epochs': 30, 'optimizer': 'SGD'}
0.881200 (0.002503) with: {'batch_size': 16, 'epochs': 30, 'optimizer': 'Adam'}
0.886567 (0.003055) with: {'batch_size': 32, 'epochs': 20, 'optimizer': 'SGD'}
0.883967 (0.002997) with: {'batch_size': 32, 'epochs': 20, 'optimizer': 'Adam'}
0.887717 (0.002917) with: {'batch_size': 32, 'epochs': 30, 'optimizer': 'SGD'}
0.883000 (0.003662) with: {'batch_size': 32, 'epochs': 30, 'optimizer': 'Adam'}


# Otimizando SGD

In [12]:
def create_model_SGD(neurons=128, learn_rate=0.01, momentum=0):
    ann = Sequential()
    ann.add(Dense(units=neurons, activation='relu', kernel_initializer='he_normal'))
    ann.add(Dense(units=10, activation='softmax'))
    optimizer = tf.keras.optimizers.SGD(learning_rate=learn_rate, momentum=momentum)
    ann.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return ann

# create model
model = KerasClassifier(build_fn=create_model_SGD, epochs=30, batch_size=32, verbose=2)

In [13]:
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
momentum = [0.0, 0.2, 0.4]
neurons = [64, 128]
param_grid = dict(neurons=neurons, learn_rate=learn_rate, momentum=momentum)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=None, cv=5)
grid_result = grid.fit(X_train, y_train)

Epoch 1/30
1500/1500 - 2s - loss: 1.0021 - accuracy: 0.6674
Epoch 2/30
1500/1500 - 2s - loss: 0.6473 - accuracy: 0.7781
Epoch 3/30
1500/1500 - 2s - loss: 0.5688 - accuracy: 0.8024
Epoch 4/30
1500/1500 - 2s - loss: 0.5266 - accuracy: 0.8153
Epoch 5/30
1500/1500 - 2s - loss: 0.4995 - accuracy: 0.8236
Epoch 6/30
1500/1500 - 2s - loss: 0.4797 - accuracy: 0.8311
Epoch 7/30
1500/1500 - 2s - loss: 0.4645 - accuracy: 0.8362
Epoch 8/30
1500/1500 - 2s - loss: 0.4523 - accuracy: 0.8414
Epoch 9/30
1500/1500 - 2s - loss: 0.4420 - accuracy: 0.8452
Epoch 10/30
1500/1500 - 2s - loss: 0.4332 - accuracy: 0.8474
Epoch 11/30
1500/1500 - 2s - loss: 0.4254 - accuracy: 0.8501
Epoch 12/30
1500/1500 - 2s - loss: 0.4186 - accuracy: 0.8522
Epoch 13/30
1500/1500 - 2s - loss: 0.4125 - accuracy: 0.8539
Epoch 14/30
1500/1500 - 2s - loss: 0.4069 - accuracy: 0.8564
Epoch 15/30
1500/1500 - 2s - loss: 0.4016 - accuracy: 0.8585
Epoch 16/30
1500/1500 - 2s - loss: 0.3968 - accuracy: 0.8591
Epoch 17/30
1500/1500 - 2s - loss

Epoch 13/30
1500/1500 - 2s - loss: 0.4113 - accuracy: 0.8550
Epoch 14/30
1500/1500 - 2s - loss: 0.4055 - accuracy: 0.8561
Epoch 15/30
1500/1500 - 2s - loss: 0.4002 - accuracy: 0.8583
Epoch 16/30
1500/1500 - 2s - loss: 0.3951 - accuracy: 0.8599
Epoch 17/30
1500/1500 - 2s - loss: 0.3906 - accuracy: 0.8611
Epoch 18/30
1500/1500 - 2s - loss: 0.3863 - accuracy: 0.8626
Epoch 19/30
1500/1500 - 2s - loss: 0.3822 - accuracy: 0.8646
Epoch 20/30
1500/1500 - 2s - loss: 0.3785 - accuracy: 0.8660
Epoch 21/30
1500/1500 - 2s - loss: 0.3748 - accuracy: 0.8674
Epoch 22/30
1500/1500 - 2s - loss: 0.3715 - accuracy: 0.8680
Epoch 23/30
1500/1500 - 2s - loss: 0.3682 - accuracy: 0.8699
Epoch 24/30
1500/1500 - 2s - loss: 0.3650 - accuracy: 0.8699
Epoch 25/30
1500/1500 - 2s - loss: 0.3621 - accuracy: 0.8718
Epoch 26/30
1500/1500 - 2s - loss: 0.3592 - accuracy: 0.8730
Epoch 27/30
1500/1500 - 2s - loss: 0.3565 - accuracy: 0.8735
Epoch 28/30
1500/1500 - 2s - loss: 0.3539 - accuracy: 0.8746
Epoch 29/30
1500/1500 - 

Epoch 25/30
1500/1500 - 2s - loss: 0.3515 - accuracy: 0.8765
Epoch 26/30
1500/1500 - 2s - loss: 0.3486 - accuracy: 0.8780
Epoch 27/30
1500/1500 - 2s - loss: 0.3460 - accuracy: 0.8784
Epoch 28/30
1500/1500 - 2s - loss: 0.3432 - accuracy: 0.8799
Epoch 29/30
1500/1500 - 2s - loss: 0.3407 - accuracy: 0.8806
Epoch 30/30
1500/1500 - 2s - loss: 0.3383 - accuracy: 0.8810
375/375 - 0s - loss: 0.3800 - accuracy: 0.8634
Epoch 1/30
1500/1500 - 2s - loss: 0.9637 - accuracy: 0.6861
Epoch 2/30
1500/1500 - 2s - loss: 0.6275 - accuracy: 0.7852
Epoch 3/30
1500/1500 - 2s - loss: 0.5537 - accuracy: 0.8090
Epoch 4/30
1500/1500 - 2s - loss: 0.5131 - accuracy: 0.8216
Epoch 5/30
1500/1500 - 2s - loss: 0.4863 - accuracy: 0.8297
Epoch 6/30
1500/1500 - 2s - loss: 0.4669 - accuracy: 0.8369
Epoch 7/30
1500/1500 - 2s - loss: 0.4518 - accuracy: 0.8405
Epoch 8/30
1500/1500 - 2s - loss: 0.4394 - accuracy: 0.8452
Epoch 9/30
1500/1500 - 2s - loss: 0.4288 - accuracy: 0.8493
Epoch 10/30
1500/1500 - 2s - loss: 0.4198 - acc

1500/1500 - 2s - loss: 0.4583 - accuracy: 0.8390
Epoch 7/30
1500/1500 - 2s - loss: 0.4442 - accuracy: 0.8430
Epoch 8/30
1500/1500 - 2s - loss: 0.4324 - accuracy: 0.8479
Epoch 9/30
1500/1500 - 2s - loss: 0.4230 - accuracy: 0.8514
Epoch 10/30
1500/1500 - 2s - loss: 0.4144 - accuracy: 0.8547
Epoch 11/30
1500/1500 - 2s - loss: 0.4072 - accuracy: 0.8568
Epoch 12/30
1500/1500 - 2s - loss: 0.4005 - accuracy: 0.8591
Epoch 13/30
1500/1500 - 2s - loss: 0.3947 - accuracy: 0.8605
Epoch 14/30
1500/1500 - 2s - loss: 0.3893 - accuracy: 0.8624
Epoch 15/30
1500/1500 - 2s - loss: 0.3843 - accuracy: 0.8646
Epoch 16/30
1500/1500 - 2s - loss: 0.3799 - accuracy: 0.8661
Epoch 17/30
1500/1500 - 2s - loss: 0.3756 - accuracy: 0.8682
Epoch 18/30
1500/1500 - 2s - loss: 0.3715 - accuracy: 0.8691
Epoch 19/30
1500/1500 - 2s - loss: 0.3677 - accuracy: 0.8702
Epoch 20/30
1500/1500 - 2s - loss: 0.3642 - accuracy: 0.8721
Epoch 21/30
1500/1500 - 2s - loss: 0.3609 - accuracy: 0.8726
Epoch 22/30
1500/1500 - 2s - loss: 0.35

1500/1500 - 2s - loss: 0.3610 - accuracy: 0.8732
Epoch 19/30
1500/1500 - 2s - loss: 0.3571 - accuracy: 0.8743
Epoch 20/30
1500/1500 - 2s - loss: 0.3533 - accuracy: 0.8754
Epoch 21/30
1500/1500 - 2s - loss: 0.3498 - accuracy: 0.8766
Epoch 22/30
1500/1500 - 2s - loss: 0.3462 - accuracy: 0.8773
Epoch 23/30
1500/1500 - 2s - loss: 0.3429 - accuracy: 0.8785
Epoch 24/30
1500/1500 - 2s - loss: 0.3400 - accuracy: 0.8798
Epoch 25/30
1500/1500 - 2s - loss: 0.3369 - accuracy: 0.8810
Epoch 26/30
1500/1500 - 2s - loss: 0.3340 - accuracy: 0.8820
Epoch 27/30
1500/1500 - 2s - loss: 0.3312 - accuracy: 0.8830
Epoch 28/30
1500/1500 - 2s - loss: 0.3286 - accuracy: 0.8840
Epoch 29/30
1500/1500 - 2s - loss: 0.3260 - accuracy: 0.8846
Epoch 30/30
1500/1500 - 2s - loss: 0.3234 - accuracy: 0.8849
375/375 - 0s - loss: 0.3585 - accuracy: 0.8746
Epoch 1/30
1500/1500 - 2s - loss: 0.9014 - accuracy: 0.6959
Epoch 2/30
1500/1500 - 2s - loss: 0.5979 - accuracy: 0.7914
Epoch 3/30
1500/1500 - 2s - loss: 0.5313 - accuracy:

...

Epoch 10/30
1500/1500 - 2s - loss: 0.2330 - accuracy: 0.9148
Epoch 11/30
1500/1500 - 2s - loss: 0.2267 - accuracy: 0.9175
Epoch 12/30
1500/1500 - 2s - loss: 0.2179 - accuracy: 0.9206
Epoch 13/30
1500/1500 - 2s - loss: 0.2095 - accuracy: 0.9237
Epoch 14/30
1500/1500 - 2s - loss: 0.2037 - accuracy: 0.9269
Epoch 15/30
1500/1500 - 2s - loss: 0.1986 - accuracy: 0.9280
Epoch 16/30
1500/1500 - 2s - loss: 0.1957 - accuracy: 0.9296
Epoch 17/30
1500/1500 - 2s - loss: 0.1902 - accuracy: 0.9299
Epoch 18/30
1500/1500 - 2s - loss: 0.1837 - accuracy: 0.9341
Epoch 19/30
1500/1500 - 2s - loss: 0.1833 - accuracy: 0.9346
Epoch 20/30
1500/1500 - 2s - loss: 0.1820 - accuracy: 0.9360
Epoch 21/30
1500/1500 - 2s - loss: 0.1816 - accuracy: 0.9347
Epoch 22/30
1500/1500 - 2s - loss: 0.1780 - accuracy: 0.9372
Epoch 23/30
1500/1500 - 2s - loss: 0.1659 - accuracy: 0.9405
Epoch 24/30
1500/1500 - 2s - loss: 0.1507 - accuracy: 0.9453
Epoch 25/30
1500/1500 - 2s - loss: 0.1581 - accuracy: 0.9442
Epoch 26/30
1500/1500 - 

Epoch 22/30
1500/1500 - 2s - loss: 0.1580 - accuracy: 0.9421
Epoch 23/30
1500/1500 - 2s - loss: 0.1596 - accuracy: 0.9429
Epoch 24/30
1500/1500 - 2s - loss: 0.1553 - accuracy: 0.9439
Epoch 25/30
1500/1500 - 2s - loss: 0.1522 - accuracy: 0.9464
Epoch 26/30
1500/1500 - 2s - loss: 0.1508 - accuracy: 0.9460
Epoch 27/30
1500/1500 - 2s - loss: 0.1438 - accuracy: 0.9488
Epoch 28/30
1500/1500 - 2s - loss: 0.1505 - accuracy: 0.9488
Epoch 29/30
1500/1500 - 2s - loss: 0.1379 - accuracy: 0.9501
Epoch 30/30
1500/1500 - 2s - loss: 0.1307 - accuracy: 0.9526
375/375 - 0s - loss: 0.6821 - accuracy: 0.8695
Epoch 1/30
1500/1500 - 2s - loss: 0.4923 - accuracy: 0.8277
Epoch 2/30
1500/1500 - 2s - loss: 0.3750 - accuracy: 0.8672
Epoch 3/30
1500/1500 - 2s - loss: 0.3345 - accuracy: 0.8810
Epoch 4/30
1500/1500 - 2s - loss: 0.3094 - accuracy: 0.8874
Epoch 5/30
1500/1500 - 2s - loss: 0.2850 - accuracy: 0.8975
Epoch 6/30
1500/1500 - 2s - loss: 0.2699 - accuracy: 0.9029
Epoch 7/30
1500/1500 - 2s - loss: 0.2498 - a

1500/1500 - 2s - loss: 0.3266 - accuracy: 0.8796
Epoch 4/30
1500/1500 - 2s - loss: 0.3027 - accuracy: 0.8901
Epoch 5/30
1500/1500 - 2s - loss: 0.2838 - accuracy: 0.8960
Epoch 6/30
1500/1500 - 2s - loss: 0.2632 - accuracy: 0.9035
Epoch 7/30
1500/1500 - 2s - loss: 0.2513 - accuracy: 0.9094
Epoch 8/30
1500/1500 - 2s - loss: 0.2302 - accuracy: 0.9173
Epoch 9/30
1500/1500 - 2s - loss: 0.2184 - accuracy: 0.9209
Epoch 10/30
1500/1500 - 2s - loss: 0.2103 - accuracy: 0.9245
Epoch 11/30
1500/1500 - 2s - loss: 0.2047 - accuracy: 0.9265
Epoch 12/30
1500/1500 - 2s - loss: 0.1892 - accuracy: 0.9331
Epoch 13/30
1500/1500 - 2s - loss: 0.1817 - accuracy: 0.9345
Epoch 14/30
1500/1500 - 2s - loss: 0.1686 - accuracy: 0.9398
Epoch 15/30
1500/1500 - 2s - loss: 0.1634 - accuracy: 0.9428
Epoch 16/30
1500/1500 - 2s - loss: 0.1650 - accuracy: 0.9429
Epoch 17/30
1500/1500 - 2s - loss: 0.1813 - accuracy: 0.9407
Epoch 18/30
1500/1500 - 2s - loss: 0.1729 - accuracy: 0.9425
Epoch 19/30
1500/1500 - 2s - loss: 0.1466 

1500/1500 - 2s - loss: 0.2200 - accuracy: 0.9206
Epoch 16/30
1500/1500 - 2s - loss: 0.2260 - accuracy: 0.9199
Epoch 17/30
1500/1500 - 2s - loss: 0.2152 - accuracy: 0.9225
Epoch 18/30
1500/1500 - 2s - loss: 0.2244 - accuracy: 0.9224
Epoch 19/30
1500/1500 - 2s - loss: 0.2202 - accuracy: 0.9250
Epoch 20/30
1500/1500 - 2s - loss: 0.2201 - accuracy: 0.9259
Epoch 21/30
1500/1500 - 2s - loss: 0.2188 - accuracy: 0.9277
Epoch 22/30
1500/1500 - 2s - loss: 0.2042 - accuracy: 0.9305
Epoch 23/30
1500/1500 - 2s - loss: 0.1976 - accuracy: 0.9328
Epoch 24/30
1500/1500 - 2s - loss: 0.1859 - accuracy: 0.9356
Epoch 25/30
1500/1500 - 2s - loss: 0.1859 - accuracy: 0.9379
Epoch 26/30
1500/1500 - 2s - loss: 0.1896 - accuracy: 0.9367
Epoch 27/30
1500/1500 - 2s - loss: 0.1871 - accuracy: 0.9387
Epoch 28/30
1500/1500 - 2s - loss: 0.1761 - accuracy: 0.9404
Epoch 29/30
1500/1500 - 2s - loss: 0.1806 - accuracy: 0.9402
Epoch 30/30
1500/1500 - 2s - loss: 0.1646 - accuracy: 0.9425
375/375 - 0s - loss: 0.8498 - accura

1500/1500 - 2s - loss: 0.1725 - accuracy: 0.9471
Epoch 28/30
1500/1500 - 2s - loss: 0.1589 - accuracy: 0.9507
Epoch 29/30
1500/1500 - 2s - loss: 0.1614 - accuracy: 0.9514
Epoch 30/30
1500/1500 - 2s - loss: 0.1543 - accuracy: 0.9530
375/375 - 0s - loss: 0.8789 - accuracy: 0.8722
Epoch 1/30
1500/1500 - 2s - loss: 0.5164 - accuracy: 0.8242
Epoch 2/30
1500/1500 - 2s - loss: 0.4069 - accuracy: 0.8592
Epoch 3/30
1500/1500 - 2s - loss: 0.3610 - accuracy: 0.8736
Epoch 4/30
1500/1500 - 2s - loss: 0.3328 - accuracy: 0.8821
Epoch 5/30
1500/1500 - 2s - loss: 0.3112 - accuracy: 0.8907
Epoch 6/30
1500/1500 - 2s - loss: 0.3048 - accuracy: 0.8938
Epoch 7/30
1500/1500 - 2s - loss: 0.2918 - accuracy: 0.8999
Epoch 8/30
1500/1500 - 2s - loss: 0.2696 - accuracy: 0.9050
Epoch 9/30
1500/1500 - 2s - loss: 0.2662 - accuracy: 0.9105
Epoch 10/30
1500/1500 - 2s - loss: 0.2470 - accuracy: 0.9172
Epoch 11/30
1500/1500 - 2s - loss: 0.2428 - accuracy: 0.9177
Epoch 12/30
1500/1500 - 2s - loss: 0.2443 - accuracy: 0.918

In [14]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.887050 using {'learn_rate': 0.01, 'momentum': 0.2, 'neurons': 128}
0.863717 (0.003784) with: {'learn_rate': 0.001, 'momentum': 0.0, 'neurons': 64}
0.867467 (0.003153) with: {'learn_rate': 0.001, 'momentum': 0.0, 'neurons': 128}
0.867367 (0.003488) with: {'learn_rate': 0.001, 'momentum': 0.2, 'neurons': 64}
0.870383 (0.003017) with: {'learn_rate': 0.001, 'momentum': 0.2, 'neurons': 128}
0.872000 (0.002258) with: {'learn_rate': 0.001, 'momentum': 0.4, 'neurons': 64}
0.875650 (0.003004) with: {'learn_rate': 0.001, 'momentum': 0.4, 'neurons': 128}
0.884817 (0.002908) with: {'learn_rate': 0.01, 'momentum': 0.0, 'neurons': 64}
0.886567 (0.001164) with: {'learn_rate': 0.01, 'momentum': 0.0, 'neurons': 128}
0.882517 (0.003213) with: {'learn_rate': 0.01, 'momentum': 0.2, 'neurons': 64}
0.887050 (0.002101) with: {'learn_rate': 0.01, 'momentum': 0.2, 'neurons': 128}
0.882550 (0.000897) with: {'learn_rate': 0.01, 'momentum': 0.4, 'neurons': 64}
0.886133 (0.001937) with: {'learn_rate': 0.01

# Save best model
https://machinelearningmastery.com/save-load-keras-deep-learning-models/

In [15]:
best_model = Sequential()
best_model.add(Dense(units=128, activation='relu', kernel_initializer='he_normal'))
best_model.add(Dense(units=10, activation='softmax'))

opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0)
best_model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = best_model.fit(X_train, y_train, epochs=30, batch_size=32)

Epoch 1/30
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5302 - accuracy: 0.8144
Epoch 2/30
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3879 - accuracy: 0.8622
Epoch 3/30
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3518 - accuracy: 0.8749
Epoch 4/30
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3293 - accuracy: 0.8830
Epoch 5/30
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3120 - accuracy: 0.8881
Epoch 6/30
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2975 - accuracy: 0.8936
Epoch 7/30
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2852 - accuracy: 0.8984
Epoch 8/30
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2751 - accuracy: 0.9027
Epoch 9/30
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2652 - accuracy: 0.9057
Epoch 10/30
1875/1875 [==============================] - 3s 2ms/step - loss: 0.255

In [16]:
loss, accuracy = best_model.evaluate(X_test, y_test)
print('Loss: %.2f' % loss)
print('Accuracy: %.2f' % (accuracy*100))

313/313 [==============================] - 1s 2ms/step - loss: 0.3213 - accuracy: 0.8909
Loss: 0.32
Accuracy: 89.09


In [17]:
best_model.predict(X_test)[0]

array([9.93551016e-01, 9.93857885e-09, 3.59484955e-04, 1.04547462e-05,
       2.05208636e-07, 1.34142211e-10, 6.06917031e-03, 1.40890535e-08,
       9.67597862e-06, 6.45798193e-10], dtype=float32)

In [18]:
model_json = best_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

best_model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


# Load model

In [19]:
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")

print("Loaded model from disk")

Loaded model from disk
